In [15]:
import numpy as np
import pylab as pl
from math import pi
from scipy.sparse import diags

In [16]:
# Set problem parameters/functions
kappa = 1.0  # diffusion constant
L=1.0         # length of spatial domain
T=0.5  
c= 1            # total time to solve for

In [17]:
def u_I(x):
    # initial temperature distribution
    y = np.sin(pi*x/L)
    return y

In [18]:
# Set numerical parameters
mx = 10 
my = 10            # number of gridpoints in space
mt = 1000  # number of gridpoints in time

In [19]:
# Set up the numerical environment variables
x = np.linspace(0, L, mx+1) 
y = np.linspace(0, L, my+1)                            # mesh points in space
t = np.linspace(0, T, mt+1)     # mesh points in time
deltax = x[1] - x[0]  
deltay = y[1] - y[0]
# gridspacing in x
deltat = t[1] - t[0]            # gridspacing in t
lmbda_x = kappa*deltat/(deltax**2) 
lmbda_y = kappa*deltat/(deltay**2)# mesh fourier number
print("deltax=",deltax)
print("deltat=",deltat)
print("lambda=",lmbda)

deltax= 0.1
deltat= 0.0005
lambda= 0.04999999999999999


In [6]:
#Wave equation solution
def u(x,t):
    return 0.5*(np.sin(x+c*t) + np.sin(x-c*t))

In [20]:
# Set up the solution variables
u_k = np.zeros(x.size)        # u at current time step
u_kp1 = np.zeros(x.size)      # u at next time step

In [21]:
def rhs_function(A, u_j,F_j,j):
    u_jp1 = np.matmul(A , u_j) + deltat*F_j(x,t[j])
    u_j = u_jp1
    return u_j
    

In [22]:
def periodic_boundary(A, u_j):
    u_jp1 = np.matmul(A , u_j)
    u_j = u_jp1
    return u_j

In [23]:
#def dirichlet_boundary(A,s,u_j,u_jp1):
def dirichlet_boundary(A, u_j,s,u_jp1,args):
    p= args[0] 
    q = args[1]                   
    u_jp1[1:-1] = np.matmul(A , u_j[1:-1]).reshape(9,1) + lmbda*s
    u_jp1[0] = p(t[j+1]); u_jp1[mx] = q(t[j+1])
    u_j = u_jp1
    return  u_j

In [24]:
def Neumann_boundary(A,s,u_j,args):
    P = args[0]
    Q = args[1]
    u_jp1 = np.matmul(A , u_j).reshape(11,1) + 2*lmbda*deltax*s
    u_j = u_jp1
    return u_j

In [63]:
u_kp1 = np.zeros((mx+1,1))
m = [lmbda_y*np.ones(my-2),(1-2*(lmbda_x+lmbda_y))*np.ones(my-1),lmbda_y*np.ones(my-2)]
offset = [-1,0,1]
A = diags(m,offset).toarray()
h = [np.zeros(my-2),lmbda_x*np.ones(my-1),np.zeros(my-2)]
B = diags(h,offset).toarray()
    
import numpy as np
from scipy.linalg import block_diag

def tridiag(c, u, d, N): 
    # c, u, d are center, upper and lower blocks, repeat N times
    cc = block_diag(*([c]*N)) 
    shift = c.shape[1]
    uu = block_diag(*([u]*N)) 
    uu = np.hstack((np.zeros((uu.shape[0], shift)), uu[:,:-shift]))
    dd = block_diag(*([d]*N)) 
    dd = np.hstack((dd[:,shift:],np.zeros((uu.shape[0], shift))))
    return cc+uu+dd


N =my-1
AFE2= tridiag(A,B,B,N)

In [71]:
# function to solve PDE using euler method
def PDE_solve_euler(mt,mx,boundary_condition, initial_condition, AFE2):
    u_kx = np.zeros(x.size) 
    u_ky = np.zeros(y.size)
    u_k = np.zeros((x.size, y.size))                        # u at current time step
    u_kp1 = np.zeros(x.size)      # u at next time step
    # u at next time step
    # Set initial condition
    for i in range(0, mx+1):
        u_k[i] = initial_condition(x[i])
    
    for k in range(0, mt):
        u_kp1[1:-1] = np.matmul(AFE2 , u_k[1:-1])
        u_k = u_kp1
    
    return u_j

In [72]:
PDE_solve_euler(1000,10,[0,0],u_I,AFE2)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 9 is different from 81)

In [65]:
# Plot the final result and exact solution for forward euler
pl.plot(x,PDE_solve_euler(1000,10,[0,0],u_I,AFE2),'ro',label='num')
#xx = np.linspace(0,L,250)
#pl.plot(xx,u_exact(xx,T),'b-',label='exact')
pl.xlabel('x')
pl.ylabel('u(x,0.5)')
pl.legend(loc='upper right')
pl.show

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 9 is different from 81)